In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
import tqdm
from matplotlib.pyplot import figure
import torch
import torch.nn as nn
import torchvision as tv
%matplotlib inline

## Implementation of own convolution

In [26]:
def my_conv2d(x, kernel, stride=(1,1), padding=(0,0), bias=None):
    """
    Arguments:
    :param  x: input tensor 4d, type tensor.FloatTensor, BxCxHxW,
    :param  kernel: input kernel tensor 3d, type tensor.FloatTensor, CxHxW,
    :param  stride: tuple - stride parameters, set in HxW format,
    :param  padding:  tuple - padding parameters, set in HxW format,
    :param  bias : the input tensor bias is added to the output tensor,
    
    where:
    B is the batch or the number of input,
    C is the number of image channels,
    H is the height of image,
    W is is the width of the image.
    """
    
    # getting input parameters
    in_s = x.size()
    if x.dim() == 3:
        b_in, c_in, h_in, w_in = 1, in_s[0], in_s[1], in_s[2]
    elif x.dim() == 4:
        b_in, c_in, h_in, w_in = in_s[0], in_s[1], in_s[2], in_s[3]
    else:
        raise Exception("ERROR: Wrong input tensor. Input tensor must be 3d or 4d.")
        
    k_s = kernel.size()
    k_c, k_h, k_w = k_s[0], k_s[1], k_s[2]
    
    # output size calculation
    h_out = (h_in + 2 * padding[0] - k_h) / stride[0] + 1
    w_out = (w_in + 2 * padding[1] - k_w) / stride[1] + 1
    
    if not h_out.is_integer() or not w_out.is_integer():
        print("Wrong output dimension. H_out = {}, W_out = {}.".format(h_out, w_out))
        print("Some input units will be left out.")
        h_out = (h_in + 2 * padding[0] - k_h) // stride[0] + 1
        w_out = (w_in + 2 * padding[1] - k_w) // stride[1] + 1
    h_out = int(h_out)
    w_out = int(w_out)
    print("Output size: H_out = {}, W_out = {}".format(h_out, w_out))
        
    # adding padding
    x = torch.from_numpy(np.pad(x.numpy(),
                                ((0, 0), (padding[0],padding[0]), (padding[1],padding[1])),
                                mode='constant'))
    
    # output tensor
    x_out = torch.zeros(h_out, w_out)
    
    # TODO: SMTH WRONG WITH STRIDE. DON'T KNOW...
    # iterating over input tensor
    #for b in range(b_in):
    for c in range(c_in):
        #for h in range(0, h_in + 2 * padding[0] - k_h + 1, stride[0]):
        for h in range(h_out):
            #for w in range(0, w_in + 2 * padding[1] - k_w + 1, stride[1]):
            for w in range(w_out):
                res = 0
                for kc in range(k_c):
                    for kh in range(k_h):
                        for kw in range(k_w):
                            #res += x[c][h + kh][w + kw] * kernel[kc][kh][kw]
                            px += x[c][h * stride[0] + kh][w * stride[1] + kw] * kernel[kc][kh][kw]
                #x_out[int(h/stride[0])][int(w/stride[1])] += (res / k_c)
                x_out[h][w] += (px / k_c)
                            
    return x_out

## Batch normalization module

In [40]:
class BatchNorm2d(nn.Module):
    
    def __init__(self, in_channels, affine=True, beta=0.9, epsilon=1e-8):
        """       
         BatchNorm Initialization
         If the affine flag is set, then gamma and b matrices must be initialized
         to implement affine transformations in the process of training and testing.
         The parameters to be learned are set as a tensor of the corresponding dimension and save their
         self.weight
         self.bias
         
        : param in_channels: the number of input channels of the previous layer
        : param affine: whether to make affine transformation in the learning process
        : param beta: smoothing parameter (momentum)
        : param epsilon: parameter excluding division by zero
        """
        super().__init__()
        
        self.in_c = in_channels
        self.momentum = beta
        self.eps = epsilon
        
        self.gamma = torch.Tensor(1, self.in_c, 1, 1).fill_(1)
        self.b = torch.Tensor(1, self.in_c, 1, 1).fill_(0)
        
        self.mean = torch.Tensor(1, self.in_c, 1, 1).fill_(0)
        self.var = torch.Tensor(1, self.in_c, 1, 1).fill_(1)
        self.runing_mean = torch.Tensor(1, self.in_c, 1, 1).fill_(0)
        self.runing_var = torch.Tensor(1, self.in_c, 1, 1).fill_(1)
        
        
    def forward(self, x):
        """
        Считаем параметры нормализации в режиме обучения, и нормализуем x в обоих режимах
        используем для расчета параметры gamma и b КАК обучаемые, т.е. учитываем, 
        что эти параметры должны быть обучены в процессе тренировки. 
        """
        
        self.mean = torch.Tensor(1, self.in_c, 1, 1)
        self.var = torch.Tensor(1, self.in_c, 1, 1)
        for i in range(self.in_c):
            self.mean[0][i] = torch.mean(x[0][i])
            self.var[0][i] = torch.var(x[0][i])
        self.mean.expand_as(x)
        self.var.expand_as(x)
        
        if self.training:
            self.runing_mean = self.momentum * self.runing_mean.expand_as(x) + (1 - self.momentum) * self.mean
            self.runing_var = self.momentum * self.runing_var.expand_as(x) + (1 - self.momentum) * self.var
            x_norm = (x - self.runing_mean) / np.sqrt(self.runing_var + self.eps)
        else:
            x_norm = (x - self.mean) / np.sqrt(self.var + self.eps)
        
        x_hat = self.gamma * x_norm + self.b
        
        return x_hat
